In [15]:
!mkdir -p data


In [21]:
%%writefile "D:/KONOZ UNI/z programming LEVEL4/Sports Tournament Scheduling-AI/data/teams_venues_times.py"
teams = [
    "Al Ahly", "Zamalek", "Pyramids", "Masry", "Future", "Ismaily",
    "Smouha", "ENPPI", "Ceramica", "National Bank", "Talaea El Gaish",
    "Alexandria Union", "El Dakhleya", "El Gouna", "Zed",
    "Modern Sport", "Pharco", "Wadi Degla"
]

venues = [
    "Cairo Stadium", "Borg El Arab", "Air Defense Stadium",
    "Suez Stadium", "Alexandria Stadium", "Petro Sport Stadium",
    "Military Academy Stadium", "Al Salam Stadium", "El Sekka El Hadeed Stadium",
    "Zed Club Stadium"
]

match_times = ["17:00", "20:00"]


Overwriting D:/KONOZ UNI/z programming LEVEL4/Sports Tournament Scheduling-AI/data/teams_venues_times.py


In [17]:
import random
from datetime import datetime, timedelta
from data.teams_venues_times import teams, venues, match_times

# مدة البطولة
start_date = datetime(2025, 5, 1)
end_date = datetime(2026, 1, 31)

# إعداد أيام البطولة
all_dates = []
current_date = start_date
while current_date <= end_date:
    all_dates.append(current_date)
    current_date += timedelta(days=1)


# تعريف الـ Match
class Match:
    def __init__(self, team1, team2, date, time, venue):
        self.team1 = team1
        self.team2 = team2
        self.date = date
        self.time = time
        self.venue = venue

    def __repr__(self):
        return f"{self.team1} vs {self.team2} on {self.date.strftime('%Y-%m-%d')} at {self.time} in {self.venue}"


# توليد جدول دوري أسبوعي مع تكرار المباريات مرتين
def generate_weekly_schedule(teams, venues, all_dates, match_times, min_rest_days=4):
    schedule = []
    last_played = {team: start_date - timedelta(days=min_rest_days) for team in teams}

    # معلومات لكل يوم
    day_info = {d: {'count': 0, 'venues': set(), 'times': set()} for d in all_dates}

    pairings = []
    for t1 in teams:
        for t2 in teams:
            if t1 != t2:
                pairings.append((t1, t2))
                pairings.append((t2, t1))

    random.shuffle(pairings)
    matches_count = {(t1, t2): 0 for t1 in teams for t2 in teams if t1 != t2}

    week_dates = all_dates[::7]  # بداية كل أسبوع

    for week_start in week_dates:
        used_teams = set()
        for home, away in pairings:
            if matches_count[(home, away)] >= 1:
                continue
            if home in used_teams or away in used_teams:
                continue

            possible_days = [d for d in all_dates if week_start <= d < week_start + timedelta(days=7)
                             and (d - last_played[home]).days >= min_rest_days
                             and (d - last_played[away]).days >= min_rest_days
                             and day_info[d]['count'] < 2]

            if not possible_days:
                continue

            # نحاول نختار وقت وملعب متاحين
            random.shuffle(possible_days)
            for date in possible_days:
                available_times = [t for t in match_times if t not in day_info[date]['times']]
                available_venues = [v for v in venues if v not in day_info[date]['venues']]
                if not available_times or not available_venues:
                    continue

                time = random.choice(available_times)
                venue = random.choice(available_venues)

                match = Match(home, away, date, time, venue)
                schedule.append(match)

                last_played[home] = date
                last_played[away] = date
                matches_count[(home, away)] += 1
                used_teams.update([home, away])

                day_info[date]['count'] += 1
                day_info[date]['venues'].add(venue)
                day_info[date]['times'].add(time)
                break  # بعد ما نحط المباراة نخرج

    schedule.sort(key=lambda m: m.date)
    return schedule

# طباعة جدول كل فريق
def print_team_schedules(schedule):
    team_schedule = {team: [] for team in teams}
    for match in schedule:
        team_schedule[match.team1].append(match)
        team_schedule[match.team2].append(match)
    for team, matches in team_schedule.items():
        print(f"\n=== Schedule for {team} ===")
        matches.sort(key=lambda m: m.date)
        for m in matches:
            opponent = m.team2 if m.team1 == team else m.team1
            print(f"{m.date.strftime('%Y-%m-%d')} at {m.time} vs {opponent} in {m.venue}")


# تجربة الكود
if __name__ == "__main__":
    schedule = generate_weekly_schedule(teams, venues, all_dates, match_times)
    print("=== First 20 Matches in Tournament ===")
    for m in schedule[:20]:
        print(m)
    print_team_schedules(schedule)


=== First 20 Matches in Tournament ===
Masry vs El Gouna on 2025-05-01 at 20:00 in Military Academy Stadium
Future vs Ceramica on 2025-05-01 at 17:00 in Suez Stadium
Ismaily vs Modern Sport on 2025-05-03 at 17:00 in Borg El Arab
Pharco vs Talaea El Gaish on 2025-05-03 at 20:00 in Petro Sport Stadium
Wadi Degla vs National Bank on 2025-05-04 at 17:00 in Alexandria Stadium
El Dakhleya vs Alexandria Union on 2025-05-05 at 20:00 in Petro Sport Stadium
Zed vs Zamalek on 2025-05-06 at 20:00 in Suez Stadium
Pyramids vs Al Ahly on 2025-05-07 at 20:00 in Suez Stadium
Smouha vs ENPPI on 2025-05-07 at 17:00 in Borg El Arab
El Dakhleya vs Future on 2025-05-09 at 20:00 in Petro Sport Stadium
Zamalek vs National Bank on 2025-05-10 at 20:00 in Suez Stadium
Talaea El Gaish vs El Gouna on 2025-05-11 at 17:00 in Alexandria Stadium
Masry vs Ceramica on 2025-05-11 at 20:00 in Al Salam Stadium
Al Ahly vs Smouha on 2025-05-12 at 17:00 in Al Salam Stadium
Modern Sport vs Zed on 2025-05-12 at 20:00 in Alexand

In [18]:
schedule = generate_weekly_schedule(teams, venues, all_dates, match_times)
schedule[:10]  # أول 10 مباريات


[National Bank vs Wadi Degla on 2025-05-01 at 20:00 in Cairo Stadium,
 Zed vs Ceramica on 2025-05-01 at 17:00 in Alexandria Stadium,
 Modern Sport vs Pyramids on 2025-05-02 at 17:00 in Petro Sport Stadium,
 Talaea El Gaish vs El Gouna on 2025-05-04 at 20:00 in El Sekka El Hadeed Stadium,
 Masry vs Future on 2025-05-04 at 17:00 in Military Academy Stadium,
 Alexandria Union vs ENPPI on 2025-05-05 at 17:00 in Cairo Stadium,
 Pharco vs El Dakhleya on 2025-05-05 at 20:00 in El Sekka El Hadeed Stadium,
 Al Ahly vs Smouha on 2025-05-06 at 17:00 in Cairo Stadium,
 Ismaily vs Zamalek on 2025-05-07 at 17:00 in El Sekka El Hadeed Stadium,
 El Gouna vs Wadi Degla on 2025-05-08 at 17:00 in Alexandria Stadium]

In [19]:
print_team_schedules(schedule)



=== Schedule for Al Ahly ===
2025-05-06 at 17:00 vs Smouha in Cairo Stadium
2025-05-14 at 20:00 vs El Dakhleya in Al Salam Stadium
2025-05-18 at 17:00 vs Wadi Degla in Military Academy Stadium
2025-05-24 at 17:00 vs ENPPI in El Sekka El Hadeed Stadium
2025-06-04 at 20:00 vs Pyramids in Petro Sport Stadium
2025-06-11 at 20:00 vs Modern Sport in Air Defense Stadium
2025-06-17 at 20:00 vs El Gouna in Suez Stadium
2025-06-25 at 20:00 vs Talaea El Gaish in Al Salam Stadium
2025-06-30 at 20:00 vs Zamalek in El Sekka El Hadeed Stadium
2025-07-08 at 17:00 vs Talaea El Gaish in Borg El Arab
2025-07-14 at 17:00 vs El Gouna in Borg El Arab
2025-07-19 at 20:00 vs Ceramica in Cairo Stadium
2025-07-30 at 20:00 vs Ismaily in Petro Sport Stadium
2025-08-05 at 17:00 vs Alexandria Union in Alexandria Stadium
2025-08-10 at 17:00 vs ENPPI in Zed Club Stadium
2025-08-17 at 17:00 vs Pharco in Al Salam Stadium
2025-08-25 at 20:00 vs Zamalek in Cairo Stadium
2025-09-01 at 20:00 vs National Bank in El Sekka E

In [20]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Dropdown لاختيار نوع العرض
view_dropdown = widgets.Dropdown(
    options=["Full League Schedule", "Team Schedule"],
    description="View:",
    disabled=False,
)

# Dropdown لاختيار الفريق
team_dropdown = widgets.Dropdown(
    options=teams,
    description="Team:",
    disabled=False,
)

# زر عرض الجدول
button = widgets.Button(description="Show Schedule")
output = widgets.Output()

# دالة عرض الجدول
def show_schedule(b):
    output.clear_output()
    view = view_dropdown.value
    with output:
        if view == "Full League Schedule":
            html = "<h2>Full League Schedule</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Home</th><th>Away</th><th>Venue</th></tr>"
            for match in schedule:
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{match.team1}</td><td>{match.team2}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))
        else:
            team = team_dropdown.value
            team_matches = [m for m in schedule if m.team1 == team or m.team2 == team]
            html = f"<h2>Schedule for {team}</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Opponent</th><th>Venue</th></tr>"
            for match in team_matches:
                opponent = match.team2 if match.team1 == team else match.team1
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{opponent}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))

# ربط الزر بالدالة وعرض الواجهة
button.on_click(show_schedule)
display(view_dropdown, team_dropdown, button, output)


Dropdown(description='View:', options=('Full League Schedule', 'Team Schedule'), value='Full League Schedule')

Dropdown(description='Team:', options=('Al Ahly', 'Zamalek', 'Pyramids', 'Masry', 'Future', 'Ismaily', 'Smouha…

Button(description='Show Schedule', style=ButtonStyle())

Output()

In [26]:
import random
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from data.teams_venues_times import teams, venues, match_times

# مدة البطولة
start_date = datetime(2025, 5, 1)
end_date = datetime(2026, 1, 31)
all_dates = [start_date + timedelta(days=i) for i in range((end_date-start_date).days+1)]

# Match class
class Match:
    def __init__(self, team1, team2, date, time, venue):
        self.team1 = team1
        self.team2 = team2
        self.date = date
        self.time = time
        self.venue = venue
    def __repr__(self):
        return f"{self.team1} vs {self.team2} on {self.date.strftime('%Y-%m-%d')} at {self.time} in {self.venue}"

# توليد الجدول
def generate_weekly_schedule(teams_list, venues_list, all_dates, match_times_list, min_rest_days=4):
    schedule = []
    last_played = {team: start_date - timedelta(days=min_rest_days) for team in teams_list}
    day_info = {d: {'count':0, 'venues':set(), 'times':set()} for d in all_dates}
    
    pairings = []
    for t1 in teams_list:
        for t2 in teams_list:
            if t1 != t2:
                pairings.append((t1, t2))
                pairings.append((t2, t1))
    random.shuffle(pairings)
    matches_count = {(t1, t2):0 for t1 in teams_list for t2 in teams_list if t1!=t2}
    
    week_dates = all_dates[::7]
    for week_start in week_dates:
        used_teams = set()
        for home, away in pairings:
            if matches_count[(home, away)] >= 1:
                continue
            if home in used_teams or away in used_teams:
                continue
            possible_days = [d for d in all_dates if week_start <= d < week_start+timedelta(days=7)
                             and (d - last_played[home]).days >= min_rest_days
                             and (d - last_played[away]).days >= min_rest_days
                             and day_info[d]['count'] < 2]
            if not possible_days:
                continue
            random.shuffle(possible_days)
            for date in possible_days:
                available_times = [t for t in match_times_list if t not in day_info[date]['times']]
                available_venues = [v for v in venues_list if v not in day_info[date]['venues']]
                if not available_times or not available_venues:
                    continue
                time = random.choice(available_times)
                venue = random.choice(available_venues)
                match = Match(home, away, date, time, venue)
                schedule.append(match)
                last_played[home] = date
                last_played[away] = date
                matches_count[(home, away)] += 1
                used_teams.update([home, away])
                day_info[date]['count'] += 1
                day_info[date]['venues'].add(venue)
                day_info[date]['times'].add(time)
                break
    schedule.sort(key=lambda m: m.date)
    return schedule

# Output area
output = widgets.Output()

# Widgets اختيار عدد الفرق
team_count_slider = widgets.IntSlider(value=10, min=4, max=len(teams), step=1, description='Teams:')

# Widgets اختيار الملاعب
venue_checkboxes = {v: widgets.Checkbox(value=True, description=v) for v in venues}
venue_box = widgets.VBox(list(venue_checkboxes.values()))

# Widgets اختيار أوقات المباريات
time_checkboxes = {t: widgets.Checkbox(value=True, description=t) for t in match_times}
time_box = widgets.VBox(list(time_checkboxes.values()))

# Dropdown نوع العرض
view_dropdown = widgets.Dropdown(
    options=["Full League Schedule", "Team Schedule"],
    description="View:"
)

# Dropdown اختيار الفريق
team_dropdown = widgets.Dropdown(options=[])

# زر توليد الجدول
generate_button = widgets.Button(description="Generate Schedule")

# تحديث Dropdown الفرق حسب الجدول
def update_team_options(schedule):
    played_teams = sorted({match.team1 for match in schedule} | {match.team2 for match in schedule})
    team_dropdown.options = played_teams
    if played_teams:
        team_dropdown.value = played_teams[0]  # اختار أول فريق تلقائياً

# دالة عرض الجدول
def show_schedule(schedule):
    output.clear_output()
    with output:
        if view_dropdown.value == "Full League Schedule":
            html = "<h2>Full League Schedule</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Home</th><th>Away</th><th>Venue</th></tr>"
            for match in schedule:
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{match.team1}</td><td>{match.team2}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))
        else:
            team = team_dropdown.value
            team_matches = [m for m in schedule if m.team1==team or m.team2==team]
            if not team_matches:
                print(f"No matches found for {team}")
                return
            html = f"<h2>Schedule for {team}</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Opponent</th><th>Venue</th></tr>"
            for match in team_matches:
                opponent = match.team2 if match.team1==team else match.team1
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{opponent}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))

# دالة توليد الجدول حسب اختيار المستخدم
def generate_schedule(b):
    selected_count = team_count_slider.value
    selected_teams = random.sample(teams, selected_count)
    selected_venues = [v for v, cb in venue_checkboxes.items() if cb.value]
    selected_times = [t for t, cb in time_checkboxes.items() if cb.value]
    if not selected_teams or not selected_venues or not selected_times:
        output.clear_output()
        with output:
            print("Please select at least one team, venue, and time.")
        return
    global schedule
    schedule = generate_weekly_schedule(selected_teams, selected_venues, all_dates, selected_times)
    update_team_options(schedule)
    show_schedule(schedule)

generate_button.on_click(generate_schedule)

# عند تغيير نوع العرض، نعيد عرض الجدول للفريق الجديد إذا كان View = Team
def on_view_change(change):
    if 'new' in change and change['new'] == "Team Schedule" and 'schedule' in globals():
        show_schedule(schedule)

view_dropdown.observe(on_view_change, names='value')
team_dropdown.observe(lambda change: show_schedule(schedule) if 'schedule' in globals() else None, names='value')

# عرض الواجهة
display(
    widgets.Label("Select number of teams:"),
    team_count_slider,
    widgets.Label("Select available venues:"),
    venue_box,
    widgets.Label("Select match times:"),
    time_box,
    generate_button,
    view_dropdown,
    team_dropdown,
    output
)


Label(value='Select number of teams:')

IntSlider(value=10, description='Teams:', max=18, min=4)

Label(value='Select available venues:')

Label(value='Select match times:')

Button(description='Generate Schedule', style=ButtonStyle())

Dropdown(description='View:', options=('Full League Schedule', 'Team Schedule'), value='Full League Schedule')

Dropdown(options=(), value=None)

Output()